# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read in the csv exported from weatherPy.ipynb
cities_df = pd.read_csv("../output/cities.csv")
cities_df.head(5)

,Number,City,Country,Latitude,Longitude,Date,Max Temp (C),Cloudiness %,Humidity %,Wind Speed (kph)
0,1,Port Elizabeth,ZA,-33.92,25.57,1579821494,19.00,11,77,5.70
1,2,Avarua,CK,-21.21,-159.78,1579821494,28.00,90,88,4.10
2,3,Severo-Kuril'sk,RU,50.68,156.12,1579821232,-8.32,82,82,3.71
3,4,Albany,US,42.60,-73.97,1579821494,0.00,90,58,0.42
4,5,Copiapo,CL,-27.37,-70.33,1579821453,25.00,0,47,6.20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# get city coordinates
locations = cities_df[["Latitude", "Longitude"]]

# convert humidty to float
humidity = cities_df["Humidity %"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)


# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# create a subset of the cities_df layer that includes only the cities that meet my vacation criteria
hotel_df = cities_df.loc[(cities_df["Max Temp (C)"] <= 0) & (
    cities_df["Cloudiness %"] <= 50) & (
    cities_df["Humidity %"] <= 70) & (
    cities_df["Wind Speed (kph)"] <= 5), :]
hotel_df = hotel_df.dropna()
hotel_df.head(10)

,Number,City,Country,Latitude,Longitude,Date,Max Temp (C),Cloudiness %,Humidity %,Wind Speed (kph)
25,26,Novopokrovka,UA,49.84,36.55,1579821497,-3.00,29,58,5.00
75,76,Upernavik,GL,72.79,-56.15,1579821504,-21.03,37,59,3.72
142,143,Qaanaaq,GL,77.48,-69.36,1579821513,-24.25,0,60,2.42
168,169,Pangnirtung,CA,66.15,-65.71,1579821516,-31.00,20,67,1.30
264,265,Jumla,NP,29.27,82.18,1579821529,-8.73,0,53,2.92
287,288,Druzhba,UA,52.04,33.95,1579821533,-2.77,26,66,4.06
289,290,Homer,US,59.64,-151.55,1579821533,-7.00,1,52,2.10
310,311,Sherbrooke,CA,45.40,-71.90,1579821474,-1.11,20,58,2.10
406,407,Petropavlovsk-Kamchatsky,RU,53.05,158.65,1579821549,-19.00,0,58,3.00
429,430,Jinchang,CN,38.50,102.17,1579821552,-9.11,38,60,0.78


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# append an empty column to the dataframe to store the hotel name
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Number,City,Country,Latitude,Longitude,Date,Max Temp (C),Cloudiness %,Humidity %,Wind Speed (kph),Hotel Name
25,26,Novopokrovka,UA,49.84,36.55,1579821497,-3.00,29,58,5.00,
75,76,Upernavik,GL,72.79,-56.15,1579821504,-21.03,37,59,3.72,
142,143,Qaanaaq,GL,77.48,-69.36,1579821513,-24.25,0,60,2.42,
168,169,Pangnirtung,CA,66.15,-65.71,1579821516,-31.00,20,67,1.30,
264,265,Jumla,NP,29.27,82.18,1579821529,-8.73,0,53,2.92,


In [12]:
# get the base url for google apis
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up parameters for query
params = {
#     "location": f"{lat}, {lon}",
    "rankby": "prominence",
    "type": "lodging",
    "radius" : 5000,
    "key": g_key,
}

# for each row in the subset df, for through and retrieve nearest hotel name
for index, row in hotel_df.iterrows():

    # get coordinates from df
    lat = row['Latitude']
    lon = row['Longitude']

    # add location to params dict
    params['location'] = f"{lat}, {lon}"
    # Get the name of city in current row
    city = row['City']
    # assemble url and make API request
    print(f"Retrieving hotel results for {city}.")
    response = requests.get(base_url, params=params).json()

    # extract results
    results = response['results']
    # Use a try/except in the case that no results come back
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        # return the name of the nearest hotel to the location of the city
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
hotel_df.head()

Retrieving hotel results for Novopokrovka.
Closest hotel is Пудж дом.
------------
Retrieving hotel results for Upernavik.
Closest hotel is Gina's Guesthouse - Upernavik.
------------
Retrieving hotel results for Qaanaaq.
Closest hotel is Qaanaaq Hotel.
------------
Retrieving hotel results for Pangnirtung.
Closest hotel is Kilabuk Lodge.
------------
Retrieving hotel results for Jumla.
Closest hotel is Hotel Sundar Rara Karnali.
------------
Retrieving hotel results for Druzhba.
Missing field/result... skipping.
------------
Retrieving hotel results for Homer.
Closest hotel is Beluga Lake Lodge.
------------
Retrieving hotel results for Sherbrooke.
Closest hotel is Grand Times Hotel Sherbrooke.
------------
Retrieving hotel results for Petropavlovsk-Kamchatsky.
Closest hotel is ООО "Гостиничный комплекс "Петропавловск".
------------
Retrieving hotel results for Jinchang.
Closest hotel is Mingshihui Jinchang International Hotel.
------------


,Number,City,Country,Latitude,Longitude,Date,Max Temp (C),Cloudiness %,Humidity %,Wind Speed (kph),Hotel Name
25,26,Novopokrovka,UA,49.84,36.55,1579821497,-3.00,29,58,5.00,Пудж дом
75,76,Upernavik,GL,72.79,-56.15,1579821504,-21.03,37,59,3.72,Gina's Guesthouse - Upernavik
142,143,Qaanaaq,GL,77.48,-69.36,1579821513,-24.25,0,60,2.42,Qaanaaq Hotel
168,169,Pangnirtung,CA,66.15,-65.71,1579821516,-31.00,20,67,1.30,Kilabuk Lodge
264,265,Jumla,NP,29.27,82.18,1579821529,-8.73,0,53,2.92,Hotel Sundar Rara Karnali


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map
xy = [tuple(x) for x in locations.to_numpy()]
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
 
fig.add_layer(markers)
fig


# Display Map

Figure(layout=FigureLayout(height='420px'))